#### **Dynamic Simulation and Techno-Economic Analysis of a Solar-assisted Solvent Regeneration System for Carbon Capture**

#### **Purpose:** : This project implements a time-series simulation to evaluate the annual performance and financial viability of a 5 MW CSP plant for solvent regeneration designed in the TESPy framework. By integrating hourly solar data for Kano State, Nigeria, the model calculates the solar field’s thermal annual yield, storage tank dynamics, and the resulting Solar Contribution for the 5 MW reboiler load. It includes a techno-economic assessment, calculating the Cumulative Cash Flow and Payback Period.

#### **Sources:** 
- Akar, S. and Kurup, P. (2024) "Parabolic Trough Collector Cost Update for Industrial Process Heat in The United States."
- Innovation Norway. (2024). Conversion guidelines: Greenhouse gas emissions.
- Fedec, A. (2025) "Natural gas prices rise 3%."

##### **Author:** Bello Oluwatobi

##### **Last Updated:** November 7, 2025

### #1 Installing Libraries

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import urllib.request
import os

### #2 Loading and Cleaning Data

In [ ]:
# downloading the dataset
url = "https://drive.usercontent.google.com/download?id=1Emys7LFQfafGn0VqwZuaDHIBeBkeRED7&export=download&authuser=0&confirm=t" 

folder_path = "../data"
file_name = "solar-measurements_nigeria-kano_qc.csv"

file_path = os.path.join(folder_path, file_name)

# creates the data folder in the root directory
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

urllib.request.urlretrieve(url, file_path)


In [ ]:
# loading weather data from CSV file
df = pd.read_csv('../data/solar-measurements_nigeria-kano_qc.csv', skiprows=[1], encoding='iso-8859-1', engine='python')

In [ ]:
# parsing the timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)

# selecting relevant columns: Direct Normal Irradiance (DNI) and Ambient Temperature (Tamb)
cols_to_keep = ['DNI', 'Tamb']
df = df[cols_to_keep]

In [ ]:
# making sure data is numeric
df['DNI'] = pd.to_numeric(df['DNI'], errors='coerce')
df['Tamb'] = pd.to_numeric(df['Tamb'], errors='coerce')

In [ ]:
# handling missing data
df = df.dropna()

In [ ]:
# replacing negative values of DNI with zero
df[df < 0] = 0

In [ ]:
# resampling to hourly data and dropping any remaining NaN values
df_hourly = df.resample('h').mean().dropna()

In [ ]:
print(f"total simulation hours: {len(df_hourly)} = {len(df_hourly)/24:.0f} days")

In [ ]:
# determining peak DNI in the dataset
peak_dni_in_data = df_hourly['DNI'].max()
print(f"The maximum recorded DNI in the dataset is: {peak_dni_in_data} W/m2")

### #3 Setting Solar Thermal System Design Parameters

In [ ]:
# setting data derived from the 'thermal_design_point.ipynb' analysis
M_DOT_DESIGN = 78.1425    # kg/s (required flow to provide steam to the reboiler)
T_INLET_SOLAR = 125      # °C (fluid enters solar thermal field at 125°C)
T_TARGET_SOLAR = 140     # °C (fluid leaves solar thermal field at 140°C)
CP_WATER = 4.184         # kJ/kgK (specific heat capacity of water)


In [ ]:
# calculating the actual design thermal load required by the reboiler
Q_DEMAND_MW = (M_DOT_DESIGN * CP_WATER * (T_TARGET_SOLAR - T_INLET_SOLAR)) / 1000
print(f"Plant Target Load: {Q_DEMAND_MW:.2f} MW")

In [ ]:
# setting parameters for CSP plant
PEAK_DNI = 900           # W/m2 (Design Point DNI for sizing)
ETA_OPTICAL = 0.75       # optical efficiency of the collectors
ETA_THERMAL_LOSS = 0.85  # accounting for heat losses in pipes/receivers
SIZING_FACTOR = 2.5     # over-sizing factor to allow for increased production capacity

# calculating required solar field area
REQUIRED_AREA_BASE = (Q_DEMAND_MW * 1e6) / (PEAK_DNI * ETA_OPTICAL * ETA_THERMAL_LOSS)
SOLAR_FIELD_AREA = REQUIRED_AREA_BASE * SIZING_FACTOR

print(f"Required Base Area:  {REQUIRED_AREA_BASE:.2f} m2")
print(f"Final Area (Oversize Factor={SIZING_FACTOR}): {SOLAR_FIELD_AREA:.2f} m2")

### #4 Running Simulation Loop for the Plant  (Case 1: Direct Solar + Gas Backup)

In [ ]:
# initializing results storage for first case: solar only + gas backup
results_case_1 = []

# looping through each hourly data point to calculate performance
for timestamp, row in df_hourly.iterrows():

    dni = row['DNI']   # W/m2
    t_amb = row['Tamb'] # °C

    # calculating average temp and temp gradient
    avg_t = (T_INLET_SOLAR + T_TARGET_SOLAR) / 2
    delta_t = avg_t - t_amb

    # calculating efficiency based on DNI and temperature gradient
    if dni > 50: # only operate if DNI > 50 W/m2
        # using the standard parabolic trough efficiency curve
        eta = 0.75 - (0.45 * delta_t / dni)
        eta = max(0.0, eta) # efficiency cannot be negative
    else:
        eta = 0.0

    # calculating thermal power produced in MW
    q_produced_mw = (SOLAR_FIELD_AREA * dni * eta) / 1e6

    # executing the hybrid mode dispatch logic
    if q_produced_mw >= Q_DEMAND_MW:
        # using only solar power
        q_solar_used = Q_DEMAND_MW
        q_gas_backup = 0
        q_curtailed  = q_produced_mw - Q_DEMAND_MW
    else:
        # using solar + gas backup to meet demand
        q_solar_used = q_produced_mw
        q_gas_backup = Q_DEMAND_MW - q_produced_mw
        q_curtailed  = 0

    results_case_1.append({
        'Timestamp': timestamp,
        'DNI': dni,
        'Efficiency': eta,
        'Solar_Output_MW': q_solar_used,
        'Gas_Backup_MW': q_gas_backup
    })


print(len(results_case_1))    


### #5 Running Simulation Loop for the Plant  (Case 2: Solar + TES + Gas Backup)

In [ ]:
# setting the thermal storage parameters
STORAGE_CAPACITY_MWH = 60.0  # size of the thermal tank (4 hours of full 5MW load)
storage_level = 0.0          # current energy in tank (MWh)
TES_efficiency = 0.90        # 10% round-trip loss

In [ ]:
# initializing results storage for first case: solar only + gas backup
results_case_2 = []


for timestamp, row in df_hourly.iterrows():
    dni = row['DNI']
    t_amb = row['Tamb']

    # calculating average temp and temp gradient
    avg_t = (T_INLET_SOLAR + T_TARGET_SOLAR) / 2
    delta_t = avg_t - t_amb

    # calculating efficiency based on DNI and temperature gradient
    if dni > 50: # only operate if DNI > 50 W/m2
        # using the standard parabolic trough efficiency curve
        eta = 0.75 - (0.45 * delta_t / dni)
        eta = max(0.0, eta) # efficiency cannot be negative
    else:
        eta = 0.0
    
    # calculating thermal power produced in MW
    q_produced_mw = (SOLAR_FIELD_AREA * dni * eta) / 1e6


    # calculating difference between energy produced and energy needed
    q_net = q_produced_mw - Q_DEMAND_MW

    # when there is excess solar energy production
    if q_net > 0:
        
        # determining the charge capacity going into the storage
        charge_cap = min(q_net, (STORAGE_CAPACITY_MWH - storage_level) / TES_efficiency)
        storage_level += (charge_cap * TES_efficiency)
        
        # setting solar direct contribution and gas backup
        q_solar_used = Q_DEMAND_MW
        q_gas_backup = 0
    else:
        # using storage to cover energy deficit
        needed_from_storage = abs(q_net)
        # determining discharge amount from the storage tank
        discharge_power = min(needed_from_storage, storage_level)
        storage_level -= discharge_power

        # calculating solar contribution as energy produced + energy from storage
        q_solar_used = q_produced_mw + discharge_power
        
        # calculating gas backup needed
        q_gas_backup = Q_DEMAND_MW - q_solar_used

    results_case_2.append({
        'Timestamp': timestamp,
        'DNI': dni,
        'Solar_Output_MW': q_solar_used,
        'Gas_Backup_MW': q_gas_backup,
        'Storage_Level_MWh': storage_level
    })

### #6 Results Analysis and Visualization

#### #6.1 Case 1: Solar only + Gas Backup

In [ ]:
# converting the results to DataFrame for analysis
res_1_df = pd.DataFrame(results_case_1).set_index('Timestamp')

# determining the total demand and solar fraction for case 1
total_demand = Q_DEMAND_MW * len(res_1_df)
total_solar_1  = res_1_df['Solar_Output_MW'].sum()
solar_fraction_1 = (total_solar_1 / total_demand) * 100

print("-" * 30)
print(f"SIMULATION REPORT (Case 1: Direct Solar + Gas Backup)")
print("-" * 30)
print(f"Total Simulation Hours: {len(res_1_df)}")
print(f"% Solar Usage achieved: {solar_fraction_1:.2f}%")
print(f"Gas Backup required:     {(total_demand - total_solar_1):.2f} MWh")

# plotting the results for case 1
plt.figure(figsize=(12, 6))

# plotting the first 72 hours to visualize the day/night cycle
subset = res_1_df.iloc[:72]

# energy balance plot
plt.stackplot(subset.index,
              subset['Solar_Output_MW'],
              subset['Gas_Backup_MW'],
              labels=['Solar Output', 'Gas Backup'],
              colors=['#f1c40f', '#95a5a6'],
              alpha=0.8)

plt.axhline(y=Q_DEMAND_MW, color='red', linestyle='--', label='Reboiler Demand (5 MW)')
plt.ylabel('Thermal Power (MW)')
plt.xlabel('Timeline - 72 hour period', fontweight='bold')
plt.title(f'3-Day Period Graph for Direct Solar + Gas Backup System Performance (Solar Contribution: {solar_fraction_1:.1f}%)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %H:%M'))


plt.tight_layout()
plt.show()



#### #6.2 Case 2 : Solar + TES + Gas Backup

In [ ]:
# converting the results to DataFrame for analysis
res_2_df = pd.DataFrame(results_case_2).set_index('Timestamp')

# determining the total demand and solar fraction for case 2
total_demand = Q_DEMAND_MW * len(res_2_df)
total_solar_2 = res_2_df['Solar_Output_MW'].sum()
solar_fraction_2 = (total_solar_2 / total_demand) * 100


print("-" * 30)
print(f"SIMULATION REPORT (Case 2: Solar + TES + Gas Backup)")
print("-" * 30)
print(f"Total Simulation Hours: {len(res_2_df)}")
print(f"% Solar Usage achieved: {solar_fraction_2:.2f}%")
print(f"Gas Backup required:     {(total_demand - total_solar_2):.2f} MWh")

# plotting the results for case 2
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# plotting the first 72 hours to visualize the day/night cycle
subset = res_2_df.iloc[:72] # 3 Days


# top plot: energy balance
ax1.stackplot(subset.index, 
              subset['Solar_Output_MW'], 
              subset['Gas_Backup_MW'],
              labels=['Solar + Storage', 'Gas Backup'], 
              colors=['#f1c40f', '#95a5a6'],
              alpha=0.8)

ax1.set_title(f'3-Day Period Graph for Solar + TES + Gas Backup System Performance (Solar Contribution: {solar_fraction_2:.1f}%)')
ax1.axhline(y=Q_DEMAND_MW, color='red', linestyle='--')
ax1.set_ylabel('Thermal Power (MW)')
ax1.grid(True, alpha=0.3)
ax1.legend()

# bottom Plot: storage tank level
ax2.fill_between(subset.index, 0, subset['Storage_Level_MWh'], color='blue', alpha=0.3)
ax2.set_ylabel('Storage Level (MWh)')
ax2.set_xlabel('Time')
ax2.grid(True, alpha=0.3)

ax2.xaxis.set_major_locator(mdates.HourLocator(interval=6))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%a %H:%M'))

plt.xticks(rotation=45)
plt.xlabel('Timeline - 72 hour period', fontweight='bold')
plt.tight_layout()
plt.show()

### #7 Techno-Economic Analysis

In [ ]:
# setting up parameters for techno-economic analysis

# parabolic trough cost approx $162 per m2 installed
COST_PER_M2_INSTALLED = 162  # USD/m2

# Storage: Thermal Energy Storage cost approx $30 per kWh_thermal (Standard IRENA/NREL est.)
COST_PER_KWH_STORAGE = 30.0    # USD/kWh
TES_CAPACITY_MWH = 60.0        # MWh

# operational costs: setting natural gas price as $3.5/MMBtu
mmbtu_per_mwh = 3.412
GAS_PRICE_USD_MWH = mmbtu_per_mwh * 3.5 

# setting carbon tax at $50 per ton of CO2
CARBON_TAX_TON = 50.0

# setting emission factor for natural gas - tons of CO2 per MWh of gas burned
EMISSION_FACTOR_GAS = 0.227

# specifying reboiler efficiency
REBOILER_EFFICIENCY = 0.85

In [ ]:
# calculating the techno-economic values for case 1: Direct Solar + Gas Backup

# calculating the CAPEX based on parabolic trough installation
CAPEX_CASE_1 = SOLAR_FIELD_AREA * COST_PER_M2_INSTALLED

# setting OPEX as 15% of CAPEX
OPEX_ANNUAL_1 = 0.015 * CAPEX_CASE_1  # 1.5% of CAPEX

# calculating the savings on gas and ton of CO2 avoided
gas_mwh_avoided_1 = total_solar_1 / REBOILER_EFFICIENCY
annual_fuel_savings_1 = gas_mwh_avoided_1 * GAS_PRICE_USD_MWH
tons_co2_avoided_1 = gas_mwh_avoided_1 * EMISSION_FACTOR_GAS
annual_carbon_savings_1 = tons_co2_avoided_1 * CARBON_TAX_TON

# calculating the Net Annual Cash Flow
total_annual_revenue_1 = annual_fuel_savings_1 + annual_carbon_savings_1 - OPEX_ANNUAL_1

# calculating the payback period
payback_years_1 = CAPEX_CASE_1 / total_annual_revenue_1

# printing the results for case 1
print("\n" + "="*80)
print(" ECONOMIC ANALYSIS (Case 1: Direct Solar + Gas Backup)")
print("="*80)
print(f"1. Total CAPEX (Investment):   ${CAPEX_CASE_1:,.0f}")
print(f"2. Annual Fuel Savings:        ${annual_fuel_savings_1:,.0f}")
print(f"3. Annual Carbon Savings:      ${annual_carbon_savings_1:,.0f}")
print(f"4. Annual Maintenance Cost:   -${OPEX_ANNUAL_1:,.0f}")
print(f"--------------------------------------------------")
print(f"   NET ANNUAL SAVINGS:         ${total_annual_revenue_1:,.0f}")
print(f"--------------------------------------------------")
print(f"   PAYBACK PERIOD:             {payback_years_1:.1f} Years")
print(f"   CO2 AVOIDED:                {tons_co2_avoided_1:.1f} Tons/Year")
print("\n" + "="*80)


In [ ]:
# calculating the techno-economic values for case 2: Direct Solar + TES + Gas Backup

# calculating the CAPEX based on solar field + storage costs
cost_storage = TES_CAPACITY_MWH * 1000 * COST_PER_KWH_STORAGE
CAPEX_CASE_2 = CAPEX_CASE_1 + cost_storage

# setting OPEX as 15% of CAPEX
OPEX_ANNUAL_2 = 0.015 * CAPEX_CASE_2  # 1.5% of CAPEX

# calculating the savings on gas and ton of CO2 avoided
gas_mwh_avoided_2 = total_solar_2 / REBOILER_EFFICIENCY
annual_fuel_savings_2 = gas_mwh_avoided_2 * GAS_PRICE_USD_MWH
tons_co2_avoided_2 = gas_mwh_avoided_2 * EMISSION_FACTOR_GAS
annual_carbon_savings_2 = tons_co2_avoided_2 * CARBON_TAX_TON

# calculating the Net Annual Cash Flow
total_annual_revenue_2 = annual_fuel_savings_2 + annual_carbon_savings_2 - OPEX_ANNUAL_2

# calculating the payback period
payback_years_2 = CAPEX_CASE_2 / total_annual_revenue_2

# printing the results for case 2
print("\n" + "="*80)
print(" ECONOMIC ANALYSIS (Case 2: Solar + TES + Gas Backup)")
print("="*80)
print(f"1. Total CAPEX (Investment):   ${CAPEX_CASE_2:,.0f} (Includes ${cost_storage:,.0f} for TES)")
print(f"2. Annual Fuel Savings:        ${annual_fuel_savings_2:,.0f}")
print(f"3. Annual Carbon Savings:      ${annual_carbon_savings_2:,.0f}")
print(f"4. Annual Maintenance Cost:   -${OPEX_ANNUAL_2:,.0f}")
print(f"--------------------------------------------------")
print(f"   NET ANNUAL SAVINGS:         ${total_annual_revenue_2:,.0f}")
print(f"--------------------------------------------------")
print(f"   PAYBACK PERIOD:             {payback_years_2:.1f} Years")
print(f"   CO2 AVOIDED:                {tons_co2_avoided_2:.1f} Tons/Year")
print("\n" + "="*80)

In [ ]:
# visualization of cash flow for both cases

# setting the graph plot range based on expected project life i.e. 20 yrs
years = np.arange(0, 21)

# calculating the cash flow for case 1
cash_flow_1 = [-CAPEX_CASE_1] + [total_annual_revenue_1] * 20
cum_cash_flow_1 = np.cumsum(cash_flow_1)

# calculating the cash flow for case 1
cash_flow_2 = [-CAPEX_CASE_2] + [total_annual_revenue_2] * 20
cum_cash_flow_2 = np.cumsum(cash_flow_2)


# plotting the comparison plot for case 1 and case 2
plt.figure(figsize=(12, 7))

plt.plot(years, cum_cash_flow_1 / 1e6, marker='o', linestyle='--', color='blue', label=f'Case 1: Direct Solar (Payback {payback_years_1:.1f} yrs)')

plt.plot(years, cum_cash_flow_2 / 1e6, marker='o', linestyle='-', color='green', linewidth=2, label=f'Case 2: Solar + TES (Payback {payback_years_2:.1f} yrs)')

plt.axhline(y=0, color='red', linestyle=':', label='Break-Even Point')
plt.title(f'Techno-Economic Comparison: Direct Solar vs. Solar + Storage')
plt.xlabel('Years of Operation')
plt.ylabel('Cumulative Cost Savings (Million USD)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()